<a href="https://colab.research.google.com/github/pySTEPS/ERAD-nowcasting-course-2022/blob/hands-on-users/hands-on-session-users/notebooks/helper_input_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is for reading and processing the data needed for deterministic and probabilistic nowcasting notebooks in this folder.

In [ ]:
from google.colab import drive
import os
# mount the Google Drive folder
# don't attempt to remount if the drive is already mounted
if not os.path.exists("/content/mnt/MyDrive"):
  drive.mount("mnt")
%cd '/content/mnt/MyDrive/Colab Notebooks'
# run the previous notebook to configure the environment
%run helper_input_data.ipynb

Mounted at mnt
/content/mnt/MyDrive/Colab Notebooks
/content/mnt/MyDrive/Colab Notebooks
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.3 MB 5.2 MB/s 
Found existing installation: Shapely 1.8.2
Uninstalling Shapely-1.8.2:
  Successfully uninstalled Shapely-1.8.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 199 kB 5.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for shapely: filename=Shapely-1.8.4-cp37-cp37m-linux_x86_64.whl size=669441 sha256=4fd347bfa086c019f81fdcefc2a5ba5552a8e2ded21012e193861547a4dedc36
  Stored in directory: /root/.cache/pip/wheels/0b/f0/55/139a0ab07fbf91378f9b7062842aee4d4bb3566a1fc30ac7a4
Successfully built shapely
Looking in indexes: 

## Pre-processing steps
The pre-processing steps for the nowcasting exercise resemble the steps from block 2 and 3, and consist of the following:

### Apply data transformations
The data from FMI has already been imported after running the [helper_input_data](https://github.com/pySTEPS/ERAD-nowcasting-course-2022/blob/hands-on-users/hands-on-session-users/notebooks/helper_input_data.ipynb) notebook. The precip data is called `precip` and the metadata is called `metadata`.

In [ ]:
import numpy as np

from pysteps import motion
from pysteps.utils import transformation

# The precip variable consists of 16 time steps. We will use the first four
# to derive the motion field and make a nowcast, and the last twelve frames to
# validate the nowcasts (these will be the observations).
precip_for_forecast = precip[0:4]
precip_obs = precip[4:]

# When computing the optical flow, transforming the precipitation rates (mm/h)
# to dBR via the logarithmic transform shown in the previous exercise generally
# improves the reliability of the estimation.
precip_dbr, metadata_dbr = transformation.dB_transform(
    precip_for_forecast,
    metadata,
    threshold=0.1,
    zerovalue=-15.0
)

# Handling of NaN values has been explicitly implemented in Lucas-Kanade and VET,
# but not in DARTS. For this reason, we set all non-finite values to the minimum
# value before applying the optical flow.
precip_finite = precip_dbr.copy()
precip_finite[~np.isfinite(precip_finite)] = np.nanmin(precip_dbr)

print("metadata (dBR transformed) is: ", metadata_dbr)

metadata (dBR transformed) is:  {'projection': '+proj=stere  +lon_0=25E +lat_0=90N +lat_ts=60 +a=6371288 +x_0=380886.310 +y_0=3395677.920 +no_defs', 'x1': 0.0049823258887045085, 'y1': 0.009731985162943602, 'x2': 759752.2852757066, 'y2': 1225544.6588913496, 'cartesian_unit': 'm', 'xpixelsize': 1999.348106, 'ypixelsize': 1999.25718, 'yorigin': 'upper', 'institution': 'Finnish Meteorological Institute', 'accutime': 5.0, 'unit': 'mm/h', 'transform': 'dB', 'zerovalue': -15.0, 'threshold': -10.0, 'zr_a': 223.0, 'zr_b': 1.53, 'timestamps': array([datetime.datetime(2016, 9, 28, 14, 45),
       datetime.datetime(2016, 9, 28, 14, 50),
       datetime.datetime(2016, 9, 28, 14, 55),
       datetime.datetime(2016, 9, 28, 15, 0),
       datetime.datetime(2016, 9, 28, 15, 5),
       datetime.datetime(2016, 9, 28, 15, 10),
       datetime.datetime(2016, 9, 28, 15, 15),
       datetime.datetime(2016, 9, 28, 15, 20),
       datetime.datetime(2016, 9, 28, 15, 25),
       datetime.datetime(2016, 9, 28, 15

### Determine the motion field
Determine the motion field with the Lucas-Kanade method.

In [ ]:
# Set the method
method = "LK"

# Determine the motion field (here, using the four most recent precipitation fields)
oflow = motion.get_method(method)
motion_field = oflow(precip_finite, verbose=False)